trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae

https://translate.yandex.net/api/v1.5/tr/translate
 ? key=<API key>
 & text=<text to translate>
 & lang=<translation direction>
 & [format=<text format>]
 & [options=<translation options>]

In [2]:
from pathlib import Path
import os
import re

import requests
from xml.etree import ElementTree

In [3]:
os.chdir('../')

In [ ]:
from data import read_sentences

In [49]:
API_KEY = ''

## Reading data

In [64]:
input_sent_semcor, sent_to_id_semcor, target_sent_semcor = read_sentences('SemCor')

In [65]:
len(input_sent_semcor)

8410

In [66]:
input_sent_temp, sent_to_id_temp, target_sent_temp = read_sentences('SemCor+OMSTI')

In [193]:
len(input_sent_temp)

48679

In [68]:
# Filter sentences

input_sent_omsti = []
sent_to_id_omsti = []
target_sent_omsti = []
for ix, sent in enumerate(input_sent_temp):
    if sent not in input_sent_semcor:
        input_sent_omsti.append(sent)
        sent_to_id_omsti.append(sent_to_id_temp[ix])
        target_sent_omsti.append(target_sent_temp[ix])

In [70]:
len(sent_to_id_omsti)

40269

## Translation

In [74]:
id_file = open('data/translation/omsti/en_pt/omsti_id.txt', 'w')
tr_file = open('data/translation/omsti/en_pt/omsti_tr.txt', 'w')
or_file = open('data/translation/omsti/en_pt/omsti_or.txt', 'w')
al_file = open('data/translation/omsti/en_pt/omsti_al.txt', 'w')

In [110]:
from tqdm import tqdm
import requests
from xml.etree import ElementTree

error_api = False
error_reg = False

for sentence in tqdm(input_sent_omsti[cont:]):
    r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                      data={'key': API_KEY,\
                            'text': sentence,\
                            'lang': 'en-pt',\
                            'options': '4'})
    i += 1
    if 'Error' in r.text:
        error_api = True
        break
    else:
        tr = re.findall('<text>(.*?)</text>',r.text)
        al = re.findall('<align>(.*?)</align>',r.text)
        if len(tr) and len(al):
            id_file.write(str(i-1) + '\n')
            tr_file.write(tr[0] + '\n')
            al_file.write(al[0] + '\n')
            or_file.write(sentence + '\n')
        else:
            error_reg = True
            break    


  0%|          | 3/5443 [00:03<1:53:21,  1.25s/it]
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/krivas/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/krivas/anaconda3/lib/python3.6/site-packages/tqdm-4.19.5-py3.6.egg/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/krivas/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 84%|████████▍ | 4578/5443 [1:34:06<17:46,  1.23s/it]


In [114]:
id_file.close()
tr_file.close()
or_file.close()
al_file.close()

## Backtranslation

In [234]:
tr_file = open('data/translation/omsti/en_pt/omsti_tr.txt', 'r')

bck_tr_file = open('data/translation/omsti/pt_en/omsti_tr_2.txt', 'w')
bck_or_file = open('data/translation/omsti/pt_en/omsti_or_2.txt', 'w')
bck_al_file = open('data/translation/omsti/pt_en/omsti_al_2.txt', 'w')

In [215]:
input_sentences = []
for line in tr_file:
    input_sentences.append(line.replace('\n', ''))

In [259]:
error_api = False
error_reg = False

for sentence in tqdm(input_sentences[cont:]):
    r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                      data={'key': API_KEY,\
                            'text': sentence.replace("\n", ""),\
                            'lang': 'pt-en',\
                            'options': '4'})
    
    if '<Error' in r.text:
        error_api = True
        break
    else:
        tr = re.findall('<text>(.*?)</text>',r.text)
        al = re.findall('<align>(.*?)</align>',r.text)
        if len(tr) and len(al):
            bck_tr_file.write(tr[0] + '\n')
            bck_al_file.write(al[0] + '\n')
            bck_or_file.write(sentence + '\n')
        else:
            error_reg = True
            break
          


  1%|          | 7/712 [00:08<14:04,  1.20s/it]
Exception in thread Thread-24:
Traceback (most recent call last):
  File "/home/krivas/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/krivas/anaconda3/lib/python3.6/site-packages/tqdm-4.19.5-py3.6.egg/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/krivas/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 712/712 [14:05<00:00,  1.19s/it]


In [260]:
tr_file.close()

bck_tr_file.close()
bck_or_file.close()
bck_al_file.close()

## Check files

In [286]:
tr_file = open('data/translation/omsti/en_pt/omsti_tr.txt', 'r')
or_file = open('data/translation/omsti/en_pt/omsti_or.txt', 'r')
bck_tr_file = open('data/translation/omsti/pt_en/omsti_tr_2.txt', 'r')
bck_or_file = open('data/translation/omsti/pt_en/omsti_or_2.txt', 'r')

In [287]:
input_or = []
for sent in or_file:
    input_or.append(sent)

input_tr = []
for sent in tr_file:
    input_tr.append(sent)


In [288]:
bck_input_or = []
for sent in bck_or_file:
    bck_input_or.append(sent)

bck_input_tr = []
for sent in bck_tr_file:
    bck_input_tr.append(sent)    

In [293]:
ix= -1
input_or[ix]

'though no longer enjoying legal authority , as was the case from 1968 to 1989 , paramilitary structures effectively were institutionalized during the previous administration through the existence of special private security and vigilante services ( previously known as convivir ) legally authorized to carry arms and perform functions of public order similar to those of the armed forces and police .\n'

In [294]:
input_tr[ix]

'embora já não mais gozando de autoridade legal , como foi o caso de 1968 a 1989 , paramilitares estruturas efetivamente foram institucionalizadas durante a administração anterior através da existência de especiais de segurança privada e de vigilante, serviços ( anteriormente conhecido como convivir ) legalmente autorizadas a portar armas e desempenham funções de ordem pública, semelhantes aos das forças armadas e da polícia .\n'

In [295]:
bck_input_or[ix]

'embora já não mais gozando de autoridade legal , como foi o caso de 1968 a 1989 , paramilitares estruturas efetivamente foram institucionalizadas durante a administração anterior através da existência de especiais de segurança privada e de vigilante, serviços ( anteriormente conhecido como convivir ) legalmente autorizadas a portar armas e desempenham funções de ordem pública, semelhantes aos das forças armadas e da polícia .\n'

In [296]:
bck_input_tr[ix]

'though no longer enjoying legal authority , as was the case from 1968 to 1989 , paramilitary structures were effectively institutionalised during the previous administration through the existence of special private security and vigilante services ( formerly known as convivir ) legally authorised to bear arms and perform functions of public order, similar to those of the armed forces and the police .\n'